# Семинар: Градиентный спуск. Задачи

In [ ]:
from typing import Iterable, List
import matplotlib.pyplot as plt
import numpy as np
import abc

In [ ]:
class BaseLoss(abc.ABC):
    """Базовый класс лосса"""

    @abc.abstractmethod
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        raise NotImplementedError

    @abc.abstractmethod
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        raise NotImplementedError

In [ ]:
class MSELoss(BaseLoss):
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        return np.mean((X.dot(w) - y)**2) # реализация формулы, написанной выше. X.dot(w) - матричное умножение
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        return (2/len(y)) * X.T.dot(X.dot(w) - y) # градиент по фомуле выше

In [ ]:
def gradient_descent(
    w_init: np.ndarray,
    X: np.ndarray,
    y: np.ndarray,
    loss: BaseLoss,
    lr: float,
    n_iterations: int = 100000,
)-> List[np.ndarray]:
    """
    Функция градиентного спуска
    :param w_init: np.ndarray размера (n_feratures,) -- начальное значение вектора весов
    :param X: np.ndarray размера (n_objects, n_features) -- матрица объекты-признаки
    :param y: np.ndarray размера (n_objects,) -- вектор правильных ответов
    :param loss: Объект подкласса BaseLoss, который умеет считать градиенты при помощи loss.calc_grad(X, y, w)
    :param lr: float -- параметр величины шага, на который нужно домножать градиент
    :param n_iterations: int -- сколько итераций делать
    :return: Список из n_iterations объектов np.ndarray размера (n_features,) -- история весов на каждом шаге
    """
    weights = [] # список, в который будем записывать значения полученных весов
    weights.append(w_init) # добавим исходное значение веса
    w = weights[0] # это исходное значение понадобится нам в первой итерации
    for i in range(n_iterations): # кол-во итераций
      grad = loss.calc_grad(X, y, w) # считаем радиент с помощью функции, написанной выше
      w = w - lr*grad # меняем значение веса w соглансо формуле одной итерации град. спуска
      weights.append(w) # наполняем список весов
    return weights


## Часть 1. Градиентный спуск (вспомним формулы)

Функционал ошибки, который мы применяем в задаче регрессии — Mean Squared Error:

$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2
$$

где $x_i$ — это $i$-ый объект датасета, $y_i$ — правильный ответ для $i$-го объекта, а $w$ — веса нашей линейной модели.

Можно показать, что для линейной модели, функционал ошибки можно записать в матричном виде следующим образом:
$$
Q(w, X, y) =\frac{1}{l} (y - Xw)^T(y-Xw)
$$
или
$$
Q(w, X, y) = \frac{1}{l} || Xw - y ||^2
$$

где $X$ — это матрица объекты-признаки, а $y$ — вектор правильных ответов

Для того чтобы воспользоваться методом градиентного спуска, нам нужно посчитать градиент нашего функционала. Для MSE он будет выглядеть так:

$$
\nabla_w Q(w, X, y) = \frac{2}{l} X^T(Xw-y)
$$

Формула для одной итерации градиентного спуска выглядит следующим образом:

$$
w^t = w^{t-1} - \eta \nabla_{w} Q(w^{t-1}, X, y)
$$

Где $w^t$ — значение вектора весов на $t$-ой итерации, а $\eta$ — параметр learning rate, отвечающий за размер шага.

## Часть 2. Линейная регрессия (продолжение части 1 - используются решения из части 1).



Создадим класс для линейной регрессии. Он будет использовать интерфейс, знакомый нам из библиотеки `sklearn`.

В методе `fit` мы будем подбирать веса `w` при помощи градиентного спуска нашим методом `gradient_descent`.

В методе `predict` мы будем применять нашу регрессию к датасету,

**Задание 2.1:** Допишите код в методах `fit` и `predict` класса `LinearRegression_1`

В методе `fit` вам нужно инициализировать веса `w` (например, из члучайного распределения), применить наш `gradient_descent` и сохранить последние веса `w` из траектории.

В методе `predict` вам нужно применить линейную регрессию и вернуть вектор ответов.

Обратите внимание, что объект лосса (функционала ошибки) передаётся в момент инициализации и хранится в `self.loss`. Его нужно использовать в `fit` для `gradient_descent` (например, с `n_iterations: int = 1000`).

In [ ]:
class LinearRegression_1:
    def __init__(self, loss: BaseLoss, lr: float = 0.01, n_iterations: int = 10000, n_features: int = 2) -> None: # добавили необходимые аргументы в параметры функции
        #loss - функционал ошибки
        #lr - градиентный шаг
        self.loss = loss
        self.lr = lr
        self.n_iterations = n_iterations
        self.n_features = n_features

    def fit(self, X: np.ndarray, y: np.ndarray) -> "LinearRegression_1":
        X = np.asarray(X)
        y = np.asarray(y)
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])

        w_init = np.random.uniform(-2, 2, (self.n_features))
        self.w = (gradient_descent(w_init, X, y, self.loss, self.lr, self.n_iterations))[-1] # берем последний вес из градиентного спуска

        return self.w


    def predict(self, X: np.ndarray) -> np.ndarray:
        # Проверяем, что регрессия обучена, то есть, что был вызван fit и в нём был установлен атрибут self.w
        assert hasattr(self, "w"), "Linear regression must be fitted first"
        # Добавляем столбец из единиц для константного признака
        X = np.hstack([X, np.ones([X.shape[0], 1])])
        y_pred = X.dot(self.w)
        return y_pred # возвращаем предсказание


Класс линейной регрессии создан. Более того, мы можем управлять тем, какой функционал ошибки мы оптимизируем, просто передавая разные классы в параметр `loss` при инициализации.



Будем применять нашу регрессию на реальном датасете. Загрузим датасет с машинами (см. семинар 5_sem-sklearn-knn.ipynb):

In [ ]:
import pandas as pd

X_raw = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
    header=None,      #в исх. таблице нет названий колонок
    na_values=["?"],  #если ?, то NaN
)

X_raw.head()
X_raw = X_raw[~X_raw[25].isna()].reset_index(drop=True)

In [ ]:
y = X_raw[25] # целевой признак
X_raw = X_raw.drop(25, axis=1) # остальные признаки
y.shape

(201,)

In [ ]:
X_raw.shape

(201, 25)

**Задание 2.2:** Обработайте датасет нужными методами, чтобы на нём можно было обучать линейную регрессию (см. семинар 5_sem-sklearn-knn.ipynb):

* Заполните пропуски средними (библиотека SimpleImputer)
* Переведите категориальные признаки в числовые (в методе get_dummies использовать drop_first=True.)
* Разделите датасет на обучающую и тестовую выборку (задать: доля тестовой выборки равна 0.3, `random_state=42`, `shuffle=True`)
* Нормализуйте числовые признаки (при помощи бибилиотеки StandardScaler)

In [ ]:
np.mean(X_raw.isna())  # есть пропуски

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


0     0.00000
1     0.18408
2     0.00000
3     0.00000
4     0.00000
5     0.00995
6     0.00000
7     0.00000
8     0.00000
9     0.00000
10    0.00000
11    0.00000
12    0.00000
13    0.00000
14    0.00000
15    0.00000
16    0.00000
17    0.00000
18    0.01990
19    0.01990
20    0.00000
21    0.00995
22    0.00995
23    0.00000
24    0.00000
dtype: float64

In [ ]:
X_raw.isnull().sum() # кол-во пропусков по столбцам

0      0
1     37
2      0
3      0
4      0
5      2
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     4
19     4
20     0
21     2
22     2
23     0
24     0
dtype: int64

In [ ]:
X_raw.dtypes # типы данных в столбцах

0       int64
1     float64
2      object
3      object
4      object
5      object
6      object
7      object
8      object
9     float64
10    float64
11    float64
12    float64
13      int64
14     object
15     object
16      int64
17     object
18    float64
19    float64
20    float64
21    float64
22    float64
23      int64
24      int64
dtype: object

In [ ]:
obj_ind = X_raw.columns[(X_raw.dtypes == "object").values] # индексы столбцов, в которых хаписаны категориальные данные
real_ind = X_raw.columns[(X_raw.dtypes != "object").values] # индексы столбцов, в которых хаписаны вещественные данные
X_obj = X_raw[obj_ind] # таблица ТОЛЬКО вещественных данных
X_real = X_raw[real_ind] # таблица ТОЛЬКО числовых данных
X_obj.isna().sum() # проверка кол-ва пропусков в категориальных данных

2     0
3     0
4     0
5     2
6     0
7     0
8     0
14    0
15    0
17    0
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer # инструмент для заполнения пропусков числовых данных

In [ ]:
mis_replacer = SimpleImputer(strategy="mean") # указали, что заполнять будем именно средним

X_obj = X_obj.fillna("") # пропуски в категориальных даных булем заполнять пустыми строками
X_obj.isna().sum() # проверяем, что пропусков больше нет

2     0
3     0
4     0
5     0
6     0
7     0
8     0
14    0
15    0
17    0
dtype: int64

In [ ]:
X_no_mis_real = pd.DataFrame(
    data=mis_replacer.fit_transform(X_real), columns=X_real.columns # заполняем пропуски средним значением
)
X_no_mis_real.isna().sum() # проверяем, что пропусков больше нет


0     0
1     0
9     0
10    0
11    0
12    0
13    0
16    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
dtype: int64

In [ ]:
X_no_mis = pd.concat([X_no_mis_real, X_obj], axis=1) # склеиваем наши таблицы без пропусков
X_no_mis.head()
X_no_mis.shape

(201, 25)

In [ ]:
X_dum = pd.get_dummies(X_no_mis, drop_first=False) # переводим категориальные признаки в вещественные с помощью get_dummies
print(f"Data shape: {X_dum.shape}")
X_dum.head()

Data shape: (201, 75)


,0,1,9,10,11,12,13,16,18,19,...,15_twelve,15_two,17_1bbl,17_2bbl,17_4bbl,17_idi,17_mfi,17_mpfi,17_spdi,17_spfi
0,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
1,3.0,122.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,...,0,0,0,0,0,0,0,1,0,0
2,1.0,122.0,94.5,171.2,65.5,52.4,2823.0,152.0,2.68,3.47,...,0,0,0,0,0,0,0,1,0,0
3,2.0,164.0,99.8,176.6,66.2,54.3,2337.0,109.0,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0
4,2.0,164.0,99.4,176.6,66.4,54.3,2824.0,136.0,3.19,3.40,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split # импортируем инструмент для разделения объектов на тестовую и основную выборки

X_train, X_test, y_train, y_test = train_test_split(X_dum, y, test_size=0.3, random_state=42) # разбиваем данные на основную (train)
# и тестовую (test) выборки. Размер тестовой - test_size=0.3, по усл., обязательно утсанавливаем random_state=42

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(140, 75) (140,)
(61, 75) (61,)


In [ ]:
from sklearn.preprocessing import StandardScaler # инструмент для нормализации значений

In [ ]:
X_train.columns # некоторые из названий солбцов являются просто числами

Index([                0,                 1,                 9,
                      10,                11,                12,
                      13,                16,                18,
                      19,                20,                21,
                      22,                23,                24,
         '2_alfa-romero',          '2_audi',           '2_bmw',
           '2_chevrolet',         '2_dodge',         '2_honda',
               '2_isuzu',        '2_jaguar',         '2_mazda',
       '2_mercedes-benz',       '2_mercury',    '2_mitsubishi',
              '2_nissan',        '2_peugot',      '2_plymouth',
             '2_porsche',       '2_renault',          '2_saab',
              '2_subaru',        '2_toyota',    '2_volkswagen',
               '2_volvo',        '3_diesel',           '3_gas',
                 '4_std',         '4_turbo',              '5_',
                '5_four',           '5_two',   '6_convertible',
             '6_hardtop',     '6_hatchba

In [ ]:
X_train.columns = X_train.columns.astype(str)  # поэтому мы переводим все названия в строковый тип
X_test.columns = X_test.columns.astype(str)

In [ ]:
scaler = StandardScaler() #создаем стандартный scaler
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train) # получаем преобразованные, нормализованные данные
scaled_X_test = scaler.transform(X_test)

In [ ]:
print(scaled_X_train.shape, y_train.shape)
print(scaled_X_test.shape, y_test.shape)

(140, 75) (140,)
(61, 75) (61,)


**Задание 2.3:** Обучите написанную вами линейную регрессию на обучающей выборке

In [ ]:
class BaseLoss(abc.ABC):
    """Базовый класс лосса"""

    @abc.abstractmethod
    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: число -- значения функции потерь
        """
        raise NotImplementedError

    @abc.abstractmethod
    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :return: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        raise NotImplementedError

In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Создаем объект линейной регрессии для `MSELoss`:

In [ ]:
lr_1 = LinearRegression_1(MSELoss(), 0.01, 1000, X_train.shape[1]+1) # создавем элемент класса LinearRegression_1

(140, 75) (140,)


In [ ]:
lr_1.fit(scaled_X_train, y_train) # обучаем модель

array([ 2.22670914e+01,  3.10038174e+01,  6.56976166e+02, -9.48943417e+01,
        9.31084535e+02, -2.00911646e+02,  1.39176033e+03,  8.83103296e+02,
       -2.50449507e+02, -3.59564371e+02,  3.59935455e+01,  7.56162865e+02,
        3.34695575e+02, -4.75078840e+02,  8.15140885e+01,  1.28764345e+02,
        4.67587802e+02,  1.03653475e+03,  1.81357628e+02, -3.16690769e+02,
        2.24102979e+02, -3.74962888e+02,  9.09517147e+02,  4.82213482e+01,
        4.27055873e+02, -2.28783494e+02, -2.34598690e+02,  1.06451587e+01,
       -3.42433600e+02, -3.21396593e+02,  4.30735261e+02, -9.10830378e+01,
        2.32639126e+02, -1.74597182e+02, -4.67876139e+02, -1.70899924e+02,
       -1.86007450e+02,  1.34661754e+02, -1.35399089e+02, -3.68943094e+02,
        3.69307236e+02, -1.16041516e+02,  2.17855422e+02, -1.97959960e+02,
        5.99728430e+02, -6.73943619e+01,  4.76401180e+01, -3.67659359e+01,
       -2.59764764e+02, -6.31777260e+01, -4.17550762e+02,  4.50688626e+02,
       -4.31108913e+02,  

**Задание 2.4:** Посчитайте ошибку обученной регрессии на обучающей и тестовой выборке при помощи методов `mean_squared_error` и `r2_score` из `sklearn.metrics`.

__Коэффициент детерминации $R^2$__ показывает долю дисперсии в целевой переменной, которая объяснена зависимыми переменными.   ($R^2$ можно интерпретировать как некоторого рода нормированное MSE).

$$R^2(a, X, Y) = 1 - \frac {\sum^L_{i=1}(a(x_i) - y_i)^2}{\sum^L_{i=1}(y_i - \bar{y})^2}$$ или
$$R^2(a, X, Y) = 1 - \frac {\frac{1}{l}\sum^l_{i=1}(a(x_i) - y_i)^2}{\frac{1}{l}\sum^l_{i=1}(y_i - \bar{y})^2}$$
- Если $R^2 < 0$, значит наша модель даёт предсказание хуже константы в виде среднего значения целевой переменной, то есть абсолютно бесполезна с точки зрения MSE.
- Если $R^2 = 0$, значит мы предсказываем не лучше и не хуже константы в виде среднего значения целевой переменной.
- Если $0 < R^2 < 1$, значит модель работает лучше константного предсказания с точки зрения MSE.
- Если $R^2 = 1$, значит ошибка MSE равна нулю.


In [ ]:
y_train_pred = lr_1.predict(scaled_X_train) # строим предсказания для основной и тестовой выборки
y_pred_test = lr_1.predict(scaled_X_test)

print(f"Train MSE Error: {mean_squared_error(y_train, y_train_pred)**0.5}") # находим среднеквадратическую ошиибку (не забываем извлечь квадрат)
print(f"R^2_score for train: {r2_score(y_train, y_train_pred)}")


print(f"Test MSE Error: {mean_squared_error(y_test, y_pred_test)**0.5}")
print(f"R^2_score for test: {r2_score(y_test, y_pred_test)}")



Train MSE Error: 1236.908736713067
R^2_score for train: 0.9682456805693624
Test MSE Error: 2854.1267313780927
R^2_score for test: 0.9140637525437691


Добавим к модели L2 регуляризацию (для борьбы с переобучением). Для этого нам нужно создать новый класс для нового функционала ошибки и его градиента.

Формула функционала ошибки для MSE с L2 регуляризацией выглядит так:
$$
Q(w, X, y) = \frac{1}{\ell} \sum\limits_{i=1}^\ell (\langle x_i, w \rangle - y_i)^2 + \lambda ||w||^2
$$

Или в матричном виде:

$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2 + \lambda ||w||^2,
$$

где $\lambda$ — коэффициент регуляризации

Заметим, что (удобно для вычислений):
$$
Q(w, X, y) = \frac{1}{\ell} || Xw - y ||^2+\lambda ||w||^2 =\frac{1}{l} (y - Xw)^T(y-Xw)+ \lambda w^Tw.
$$

Градиент $\nabla_w Q(w, X, y)$ выглядит так:

$$
\nabla_w Q(w, X, y) = \frac{2}{\ell} X^T(Xw-y) + 2 \lambda w
$$

**Задание 2.5:** Реализуйте класс `MSEL2Loss`

Он должен вычислять значение функционала ошибки (лосс) $
Q(w, X, y)$ и его градиент $\nabla_w Q(w, X, y)$ по формулам (выше).

Подсказка: обратите внимание, что последний элемент вектора `w` — это bias (сдвиг) (в классе `LinearRegression` к матрице `X` добавляется колонка из единиц — константный признак). bias регуляризовать не нужно. Поэтому не забудьте убрать последний элемент из `w` при подсчёте слагаемого $\lambda||w||^2$ в `calc_loss` и занулить его при подсчёте слагаемого $2 \lambda w$ в `calc_grad`

In [ ]:
class MSEL2Loss(BaseLoss):
    def __init__(self, coef: float = 1.0):
        """
        :param coef: коэффициент регуляризации (лямбда в формуле)
        """
        self.coef = coef

    def calc_loss(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> float:
        """
        Функция для вычислений значения лосса
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета. Последний признак константный.
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии. Последний вес -- bias.
        :output: число -- значения функции потерь
        """
        # -- YOUR CODE HERE --
        # Вычислите значение функции потерь при помощи X, y и w и верните его
        return (1/len(y)) * (y - X.dot(w)).T.dot(y - X.dot(w)) + self.coef*w[:-1].T.dot(w[:-1]) # реализация формулы (берем все элементы w, кроме последнего)

    def calc_grad(self, X: np.ndarray, y: np.ndarray, w: np.ndarray) -> np.ndarray:
        """
        Функция для вычислений градиента лосса по весам w
        :param X: np.ndarray размера (n_objects, n_features) с объектами датасета
        :param y: np.ndarray размера (n_objects,) с правильными ответами
        :param w: np.ndarray размера (n_features,) с весами линейной регрессии
        :output: np.ndarray размера (n_features,) градиент функции потерь по весам w
        """
        # -- YOUR CODE HERE --
        # Вычислите значение вектора градиента при помощи X, y и w и верните его
        return (2/len(y))*X.T.dot(X.dot(w) - y) + 2*self.coef*np.append(w[:-1], 0) # берем все элменеты, кроме последнего и дробавляем в этот список 0 послденим


In [ ]:
#Проверка:
#Создадим объект лосса
losst = MSEL2Loss()

# Создадим датасет
Xt = np.arange(200).reshape(20, 10)
yt = np.arange(20)

# Создадим вектор весов
wt = np.arange(10)

#print(Xt)
#print(yt)
#print(wt)

# Выведем значение лосса и градиента на этом датасете с этим вектором весов
print(losst.calc_loss(Xt, yt, wt))
print(losst.calc_grad(Xt, yt, wt))

# Проверка, что методы реализованы правильно
assert losst.calc_loss(Xt, yt, wt) == 27410487.5, "Метод calc_loss реализован неверно"
assert np.allclose(
    losst.calc_grad(Xt, yt, wt),
    np.array(
        [
            1163180.0,
            1172283.0,
            1181386.0,
            1190489.0,
            1199592.0,
            1208695.0,
            1217798.0,
            1226901.0,
            1236004.0,
            1245089.0,
        ]
    ),
), "Метод calc_grad реализован неверно"

print("Всё верно!")

27410487.5
[1163180. 1172283. 1181386. 1190489. 1199592. 1208695. 1217798. 1226901.
 1236004. 1245089.]
Всё верно!


Теперь мы можем использовать лосс с l2 регуляризацией в нашей регрессии. Пусть:

In [ ]:
lr_1_l2 = LinearRegression_1(MSEL2Loss(0.1) , 0.01, 1000, X_train.shape[1]+1)

**Задание 2.6:** Обучите регрессию с лоссом `MSEL2Loss`. Попробуйте использовать другие коэффициенты регуляризации. Получилось ли улучшить разультат на тестовой выборке? Сравните результат применения регрессии с регуляризацией к обучающей и тестовой выборкам с результатом применения регрессии без регуляризации к тем же выборкам.(Для оценки качества использовать методы `mean_squared_error` и `r2_score` из `sklearn.metrics`).

In [ ]:
lr_1_l2.fit(scaled_X_train, y_train)

array([ 8.54049316e+01,  9.32831309e+01,  5.75065607e+02,  2.20563607e+02,
        7.51503306e+02, -9.78148446e+01,  9.16023479e+02,  8.50238783e+02,
       -7.99069429e+01, -2.54736909e+02,  5.98793118e+01,  6.47120299e+02,
        9.83811752e+01, -3.86196188e+02, -1.51827052e+02,  6.90803956e+01,
        3.58613250e+02,  9.25480647e+02,  1.48841724e+02, -2.10713911e+02,
        1.41953788e+02, -3.16340695e+02,  7.78359366e+02,  2.86714365e+01,
        5.95494796e+02, -1.42743799e+02, -1.80322102e+02, -1.16373436e+02,
       -2.96988595e+02, -2.48489110e+02,  4.13708205e+02, -1.07544610e+02,
        1.97898075e+02, -2.20272526e+02, -4.54959696e+02, -1.76548389e+02,
        4.92408115e+00,  1.07031786e+02, -1.06876570e+02, -2.91570287e+02,
        2.91385229e+02, -1.06624034e+02,  1.50084847e+02, -1.32400562e+02,
        5.06408042e+02, -6.75788341e+00, -1.78815035e+01,  2.59880147e+01,
       -2.46489875e+02, -2.75613491e+01, -3.46048021e+02,  3.63264079e+02,
       -4.13969200e+02,  

In [ ]:
# Без регуляризации:
#обучение
# self, X: np.ndarray, y: np.ndarray, n_iterations: int = 100000, n_features: int = 2

y_train_pred = lr_1.predict(scaled_X_train) #предсказание
y_pred_test = lr_1.predict(scaled_X_test)   #предсказание

print(f"Train MSE Error: {mean_squared_error(y_train, y_train_pred)**0.5}") #**0.5
print(f"R^2_score for train: {r2_score(y_train, y_train_pred)}")


print(f"Test MSE Error: {mean_squared_error(y_test, y_pred_test)**0.5}")    #**0.5
print(f"R^2_score for test: {r2_score(y_test, y_pred_test)}")

# r2_score(y_train, pred_train), r2_score(y_test, pred_test) #

Train MSE Error: 1236.908736713067
R^2_score for train: 0.9682456805693624
Test MSE Error: 2854.1267313780927
R^2_score for test: 0.9140637525437691


In [ ]:
# C регуляризацией и параметром lambda = 0.1:
#обучение
# self, X: np.ndarray, y: np.ndarray, n_iterations: int = 100000, n_features: int = 2

y_train_pred = lr_1_l2.predict(scaled_X_train) #предсказание
y_pred_test = lr_1_l2.predict(scaled_X_test)   #предсказание

print(f"Train MSE Error (reg, lambda = 0.1): {mean_squared_error(y_train, y_train_pred)**0.5}") #**0.5
print(f"R^2_score for train (reg, lambda = 0.1): {r2_score(y_train, y_train_pred)}")


print(f"Test MSE Error (reg, lambda = 0.1): {mean_squared_error(y_test, y_pred_test)**0.5}")    #**0.5
print(f"R^2_score for test (reg, lambda = 0.1): {r2_score(y_test, y_pred_test)}")

# r2_score(y_train, pred_train), r2_score(y_test, pred_test) #

Train MSE Error (reg, lambda = 0.1): 1331.9565920728419
R^2_score for train (reg, lambda = 0.1): 0.9631779775724871
Test MSE Error (reg, lambda = 0.1): 2822.1108152320444
R^2_score for test (reg, lambda = 0.1): 0.9159809037858022


In [ ]:
# C регуляризацией и параметром lambda = 0.01:
lr_1_l3 = LinearRegression_1(MSEL2Loss(0.01) , 0.01, 1000, X_train.shape[1]+1)
lr_1_l3.fit(scaled_X_train, y_train)

y_train_pred = lr_1_l3.predict(scaled_X_train) #предсказание
y_pred_test = lr_1_l3.predict(scaled_X_test)   #предсказание

print(f"Train MSE Error (reg, lambda = 0.01): {mean_squared_error(y_train, y_train_pred)**0.5}") #**0.5
print(f"R^2_score for train (reg, lambda = 0.01): {r2_score(y_train, y_train_pred)}")


print(f"Test MSE Error (reg, lambda = 0.01): {mean_squared_error(y_test, y_pred_test)**0.5}")    #**0.5
print(f"R^2_score for test (reg, lambda = 0.01): {r2_score(y_test, y_pred_test)}")

# r2_score(y_train, pred_train), r2_score(y_test, pred_test) #

Train MSE Error (reg, lambda = 0.01): 1245.0843057883862
R^2_score for train (reg, lambda = 0.01): 0.9678245216070003
Test MSE Error (reg, lambda = 0.01): 2841.361636666552
R^2_score for test (reg, lambda = 0.01): 0.914830734046163


In [ ]:
# C регуляризацией и параметром lambda = 0.11:
lr_1_l4 = LinearRegression_1(MSEL2Loss(0.11) , 0.01, 1000, X_train.shape[1]+1)
lr_1_l4.fit(scaled_X_train, y_train)

y_train_pred = lr_1_l4.predict(scaled_X_train) #предсказание
y_pred_test = lr_1_l4.predict(scaled_X_test)   #предсказание

print(f"Train MSE Error (reg, lambda = 0.11): {mean_squared_error(y_train, y_train_pred)**0.5}") #**0.5
print(f"R^2_score for train (reg, lambda = 0.11): {r2_score(y_train, y_train_pred)}")


print(f"Test MSE Error (reg, lambda = 0.11): {mean_squared_error(y_test, y_pred_test)**0.5}")    #**0.5
print(f"R^2_score for test (reg, lambda = 0.11): {r2_score(y_test, y_pred_test)}")

# r2_score(y_train, pred_train), r2_score(y_test, pred_test) #

Train MSE Error (reg, lambda = 0.11): 1341.516008266429
R^2_score for train (reg, lambda = 0.11): 0.9626475396045333
Test MSE Error (reg, lambda = 0.11): 2825.5666475570392
R^2_score for test (reg, lambda = 0.11): 0.9157750056434898


## Вывод

**Попробуйте использовать другие коэффициенты регуляризации. Получилось ли улучшить разультат на тестовой выборке? Сравните результат применения регрессии с регуляризацией к обучающей и тестовой выборкам с результатом применения регрессии без регуляризации к тем же выборкам**

Использование параметра регуляризации в линейной регрессии имеет несколько преимуществ:

1. Снижение переобучения: Параметр регуляризации помогает предотвратить переобучение модели, то есть модель, которая слишком хорошо запоминает тренировочные данные и не обобщается для новых данных. Это достигается за счет штрафа за большие значения весов модели, что приводит к более устойчивой модели.

2. Улучшение обобщающей способности: Параметр регуляризации также помогает улучшить способность модели обобщать, то есть способность модели выдавать правильные предсказания на новых данных.

3. Уменьшение мультиколлинеарности: Параметр регуляризации может помочь уменьшить мультиколлинеарность, то есть связь между признаками, что может привести к нестабильности модели.


Попробовала обучить модель с заданным павраметром регуляризации λ = 0.1, ошибка на тестовой выборке немного уменьшилась (было: `Test MSE Error: 2854.1267313780927`, стало: `Test MSE Error (reg, lambda = 0.1): 2822.1108152320444`). Ошибка на основной выборке же, наоборот, возрасла.

Если говорить про значение параметра λ = 0.01, то ошибка и на тестовой, и на тренировочной выборке увеличилась.

